In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
dataset = pd.read_csv('credit_simple.csv', sep=';')
dataset.shape #1000 linhas, 8 atributos

In [ ]:
dataset.head() #filtrar e tratar dados no código como um todo

In [10]:
y = dataset['CLASSE']
X = dataset.iloc[:,:-1]

In [ ]:
X

In [ ]:
X.isnull().sum()

In [ ]:
mediana = X['SALDO_ATUAL'].median()
mediana

In [ ]:
X['SALDO_ATUAL'].fillna(mediana, inplace=True) #tratando os valores nulos com mediana
X.isnull().sum()

In [ ]:
#entender como estado civil está preenchido, a moda é masculino solteiro

agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

In [16]:
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True) #todos os dados de estado civil que estavam nulos foram substituidos pela moda (mais repete)
X.isnull().sum()

,0
SALDO_ATUAL,0
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,0
PROPOSITO,0


In [17]:
desv = X['SALDO_ATUAL'].std() #desvio padrão do saldo atual
desv

685936688.9820067

In [18]:
X.loc[X['SALDO_ATUAL']>= 2 * desv, 'SALDO_ATUAL']

,SALDO_ATUAL
127,2.541111e+09
160,2.154441e+10


In [19]:
mediana = X['SALDO_ATUAL'].median() #encontrando a mediana dos saldos
mediana

2323.0

In [20]:
X.loc[X['SALDO_ATUAL']>= 2 * desv, 'SALDO_ATUAL'] = mediana #eliminamos os outlayers
X.loc[X['SALDO_ATUAL']>= 2 * desv]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


In [ ]:
#DATABIDING

In [21]:
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

,0
PROPOSITO,
Eletrodomésticos,12
carro novo,234
carro usado,103
educação,50
mobilia/equipamento,181
negócios,97
obras,22
outros,12
qualificação,9


In [22]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'qualificação', 'PROPOSITO'] = 'outros'

agrupado = X.groupby(['PROPOSITO']).size()
agrupado

,0
PROPOSITO,
carro novo,234
carro usado,103
educação,50
mobilia/equipamento,181
negócios,97
obras,22
outros,33
radio/tv,280


In [27]:
#extração de características

X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')

In [28]:
X['DATA']

,DATA
0,2019-01-01
1,2020-01-01
2,2020-01-02
3,2019-01-02
4,2018-01-03
...,...
995,2018-06-29
996,2018-06-30
997,2018-07-03
998,2019-07-04


In [29]:
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [30]:
X['DIASEMANA']

,DIASEMANA
0,Tuesday
1,Wednesday
2,Thursday
3,Wednesday
4,Wednesday
...,...
995,Friday
996,Saturday
997,Tuesday
998,Thursday


In [31]:
X['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [32]:
X['PROPOSITO'].unique()

array(['radio/tv', 'educação', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios', 'outros', 'obras'], dtype=object)

In [33]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

Label encoder vai transformar os atributos em dados numéricos
One Hot Encoding irá transformar as classes em colunas e classificar como True or False

In [35]:
labelencoder1 =  LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])

In [36]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


One Hot Encoding, uma coluna para cada valor

In [37]:
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [38]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS') #dataframe
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


In [39]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6
...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,3,2018,6,0
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2
997,804.0,4,38,nenhum,2018-07-03,3,7,2018,7,5
998,1845.0,4,23,nenhum,2019-07-04,3,7,2019,7,4


In [40]:
sc = StandardScaler()
m = sc.fit_transform(X.iloc[:,0:3])
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

In [42]:
X = pd.concat([X, z, pd.DataFrame(m, columns=[['SALDO_ATUAL_N', 'RESIDENCIADESDE_N', 'IDADE_N']])], axis=1)
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,...,OUTROS_stores,"(SALDO_ATUAL_N,)","(RESIDENCIADESDE_N,)","(IDADE_N,)",OUTROS_banco,OUTROS_nenhum,OUTROS_stores,"(SALDO_ATUAL_N,)","(RESIDENCIADESDE_N,)","(IDADE_N,)"
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5,...,False,-0.745516,1.046987,1.639276,False,True,False,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6,...,False,0.957740,-0.765977,-0.740241,False,True,False,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4,...,False,-0.415337,0.140505,0.687469,False,True,False,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6,...,False,1.645526,1.046987,0.475956,False,True,False,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,...,False,0.572709,1.046987,0.898982,False,True,False,0.572709,1.046987,0.898982


In [43]:
# prompt: excluir três colunas

X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'OUTROSPLANOSPGTO', 'DATA', 'OUTROS_banco'], inplace=True)


In [44]:
X #dataframe com todos tratamentos aplicados

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,"(SALDO_ATUAL_N,)","(RESIDENCIADESDE_N,)","(IDADE_N,)",OUTROS_nenhum,OUTROS_stores,"(SALDO_ATUAL_N,)","(RESIDENCIADESDE_N,)","(IDADE_N,)"
0,3,7,2019,1,5,True,False,-0.745516,1.046987,1.639276,True,False,-0.745516,1.046987,1.639276
1,0,7,2020,1,6,True,False,0.957740,-0.765977,-0.740241,True,False,0.957740,-0.765977,-0.740241
2,3,2,2020,1,4,True,False,-0.415337,0.140505,0.687469,True,False,-0.415337,0.140505,0.687469
3,3,3,2019,1,6,True,False,1.645526,1.046987,0.475956,True,False,1.645526,1.046987,0.475956
4,3,0,2018,1,6,True,False,0.572709,1.046987,0.898982,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,3,2018,6,0,True,False,-0.543562,1.046987,-0.264338,True,False,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,True,False,0.211898,1.046987,0.211566,True,False,0.211898,1.046987,0.211566
997,3,7,2018,7,5,True,False,-0.875522,1.046987,0.105809,True,False,-0.875522,1.046987,0.105809
998,3,7,2019,7,4,True,False,-0.504738,1.046987,-0.687363,True,False,-0.504738,1.046987,-0.687363
